In [1]:
from GNG_basic_model_definition import gng_basic
from keras.preprocessing.image import ImageDataGenerator
import os


2024-04-07 18:03:39.919938: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 18:03:39.919968: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 18:03:39.919991: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 18:03:39.926091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
image_folder = "/home/tanfoni/homeRepo/tanfoni/Dataset_mtcnn_split"
train_folder = image_folder + "/Train"
test_folder = image_folder + "/Test"
validation_folder = image_folder + "/Valid"


IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
BATCH_SIZE = 32


## <font size="12">Data loading</font>


In [3]:
# Get class weigts
num_real_samples = len(os.listdir(os.path.join(train_folder, 'Real')))
num_fake_samples = len(os.listdir(os.path.join(train_folder, 'Fake')))

# Calculate class weights
total_samples = num_real_samples + num_fake_samples
class_weights = {
    0: total_samples / (2 * num_real_samples),
    1: total_samples / (2 * num_fake_samples)
}

In [4]:
num_real_samples / num_fake_samples

0.2996274752154334

In [5]:
class_weights

{0: 2.1687388219004213, 1: 0.6498137376077167}

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['Real', 'Fake'])

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['Real', 'Fake'])

validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
    validation_folder,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=['Real', 'Fake'])



Found 300726 images belonging to 2 classes.
Found 86352 images belonging to 2 classes.
Found 42533 images belonging to 2 classes.


In [7]:
test_generator.class_indices

{'Real': 0, 'Fake': 1}

## <font size="12">Model definition</font>

In [8]:
mlp_layers = [240, 120, 60]
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
epochs = 10
batch_size = 32

model = gng_basic(input_shape=input_shape, mlp_units=mlp_layers, encoding_dim=480)

2024-04-07 18:03:49.340351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 867 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-04-07 18:03:49.340784: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10391 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [13]:
model.train_model_gen(train_generator, validation_generator=validation_generator, epochs=epochs, class_weights=class_weights)

Epoch 1/10
  34/9398 [..............................] - ETA: 1:42:24 - loss: 0.2466 - accuracy: 0.5285

KeyboardInterrupt: 

In [10]:
model.save_model(os.path.join(os.getcwd(),"gng_basic_model_withBackground.h5"))

Model saved to /homeRepo/tanfoni/keras_deeplab_faceseg/gng_basic_model_test_save_noTrained.h5


In [11]:
model.load_model(os.path.join(os.getcwd(),"gng_basic_model_withBackground.h5"))

Model loaded from /homeRepo/tanfoni/keras_deeplab_faceseg/gng_basic_model_test_save_noTrained.h5
